# Orientação a objetos 1

Lembretes:
* Classe: molde. Objeto: instância.
* `self`: `self` (o objeto propriamente dito) está presente em todos os métodos da classe do objeto em python.
* Duck typing: se o objeto responde à uma determinada mensagem (chamada de método) característica de um determinado tipo de objeto então esse objeto também pode ser considerado do mesmo tipo.
    * "If it looks like a duck and quacks like a duck, it's a duck".
* Restrição: não se fala em polimorfismo de sobrecarga em python. As implementações são realizadas utilizando-se valores padrões nos argumentos.
* Por padrão, os objetos herdam de `object`.

In [ ]:
class Data:
    # construtor: uma classe só tem um método construtor
    def __init__(self, dia, mes, ano):
        self.dia = dia
        self.mes = mes
        self.ano = ano
    
    # Quando precisa converter para string, chama diretamente __str__
    def __str__(self):
        return f'{self.dia}/{self.mes}/{self.ano}'


d1 = Data(1998)

print(d1)

In [ ]:
class Carro:
    def __init__(self, vel_max):
        self.vel = 0
        self.vel_max = vel_max
    
    def acelerar(self, a = 5):
        if self.vel + a > self.vel_max:
            self.vel = self.vel_max
        else:
            self.vel += a
        
        return self.vel
        
    def frear(self, a = 5):
        if self.vel - a < 0:
            self.vel = 0
        else:
            self.vel -= a
        
        return self.vel


if __name__ == '__main__':
    c1 = Carro(180)
    
    for _ in range(25):
        print(c1.acelerar(8))
        
    for _ in range(10):
        print(c1.frear(a=20))

In [ ]:
from datetime import datetime, timedelta

class TarefaNaoEncontrada(Exception):
    pass

class Projeto:
    def __init__(self, nome):
        self.nome = nome
        self.tarefas = []
    
    # Define tarefas como o iterador
    def __iter__(self):
        return iter(self.tarefas)
    
    # Sobrecarga do operador (+=)
    def __iadd__(self, tarefa):
        tarefa.dono = self
        self._add_tarefa(tarefa)
        return self
    
    # Convenção para argumento "privado"
    def _add_tarefa(self, tarefa, **kwargs):
        self.tarefas.append(tarefa)
    
    def _add_nova_tarefa(self, descricao, **kwargs):
        self.tarefas.append(Tarefa(descricao, kwargs.get('vencimento', None)))
    
    # Padrão de impressão do objeto propriamente dito
    def __str__(self):
        return f'{self.nome} ({len(self.pendentes())} tarefa(s) pendente(s))'
    
    def add(self, tarefa, vencimento=None, **kwargs):
        funcao_escolhida = self._add_tarefa if isinstance(tarefa, Tarefa) \
            else self._add_nova_tarefa
        kwargs['vencimento'] = vencimento
        funcao_escolhida(tarefa, **kwargs)
    
    def pendentes(self):
        return [tarefa for tarefa in self.tarefas if not tarefa.feito]
    
    def procurar(self, descricao):
        try:
            # Possível IndexError
            return [tarefa for tarefa in self.tarefas
                if tarefa.descricao == descricao][0]
        except IndexError as e:
            raise TarefaNaoEncontrada(str(e))


class Tarefa:
    def __init__(self, descricao, vencimento=None):
        self.descricao = descricao
        self.feito = False
        self.criacao = datetime.now()
        self.vencimento = vencimento
    
    def concluir(self):
        self.feito = True
    
    def __str__(self):
        status = []
        if self.feito:
            status.append('(Concluída)')
        elif self.vencimento:
            if datetime.now() > self.vencimento:
                status.append('(Vencida)')
            else:
                dias = (self.vencimento - datetime.now()).days
                status.append(f'(Vence em {dias} dias)')
        
        return f'{self.descricao}' + ' '.join(status)

# Herança da classe Tarefa
class TarefaRecorrente(Tarefa):
    def __init__(self, descricao, vencimento, dias=7):
        # Em python2:
        # super(TarefaRecorrente, self).__init__(descricao, vencimento)
        # Chama a função init da "classe mãe" de TarefaRecorrente
        super().__init__(descricao, vencimento)
        self.dias = dias
        self.dono = None
    
    def concluir(self):
        super().concluir()
        novo_vencimento = datetime.now() + timedelta(days=self.dias)
        nova_tarefa = TarefaRecorrente(self.descricao, novo_vencimento, self.dias)
        # Projeto
        if self.dono:
            self.dono += nova_tarefa
        return nova_tarefa
        
    
def main():
    casa = Projeto('Tarefas de Casa')
    casa.add('Passar roupa', datetime.now() + timedelta(days=3, minutes=12))
    casa.add('Lavar prato')
    casa += TarefaRecorrente('Trocar lençóis', datetime.now(), 7)
    casa.procurar('Trocar lençóis').concluir()
    print(casa)
    
    try:
        casa.procurar('Lavar prato - Erro').concluir()
    except TarefaNaoEncontrada as e:
        print(f'A causa foi "{str(e)}"!')
    finally:
        print('Sempre será executado')
    
    casa.procurar('Lavar prato').concluir()
    for tarefa in casa:
        print(f'- {tarefa}')
    print(casa)
            

if __name__ == '__main__':
    main()